In [119]:
%load_ext autoreload

%autoreload 2

from glob import glob
from os.path import join
import os
import pickle

import scipy.linalg
import scipy.stats
import matplotlib
matplotlib.rcParams["figure.dpi"] = 144
matplotlib.rcParams["savefig.dpi"] = 144
import matplotlib.pyplot as plt
import numpy as np

import subprocess
import shutil

from astropy.table import Table, Column
import astropy.units as u

import emcee

import kepler
import threebody

import minuit_wrap
%matplotlib inline

import logging
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
logging.debug("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Check everything is ready

In [120]:
!ls -l fitter_params.pickle emcee_params.pickle

-rw-r--r-- 1 aarchiba aipsuser 513 Aug  8 17:13 emcee_params.pickle
-rw-r--r-- 1 aarchiba aipsuser 439 Aug 15 10:21 fitter_params.pickle


In [113]:
shutil.copy("fitter_params.pickle","emcee_params.pickle")

In [136]:
shutil.copy("downhill-runs/lband/fitter_params.pickle","emcee_params.pickle")

In [140]:
d = pickle.load(open("emcee_params.pickle"))
F = threebody.Fitter(**d)
d

{'dmx_span': 182.62125,
 'efac': 1,
 'fdn_range': (1, 5),
 'files': 'tuned2_aligned_pulses_1400_nosun2',
 'fit_pm': True,
 'fit_pos': True,
 'fit_px': True,
 'kopeikin': False,
 'linear_dm': False,
 'linear_jumps': True,
 'parfile': '0337_de430.par',
 'ppn_mode': 'heavysimple',
 'priors': ('dbeta', 'dgamma'),
 't2_astrometry': True,
 'toa_mode': 'pipeline',
 'tzrmjd_middle': 'auto',
 'variable_dm': True,
 'variable_ipm': True}

In [141]:
F.goodness_of_fit()

36358.805574199816686

In [142]:
walker_name = "start-walkers.npy"
walkers = np.load(walker_name)
walkers.shape

(1, 142, 17)

In [143]:
if walkers.shape[-1] != len(F.parameters):
    print("Problem! Walkers don't match parameter set")
    raise ValueError
    if len(F.parameters)<walkers.shape[-1]:
        print("Trimming")
        walkers = walkers[...,:len(F.parameters)]
        np.save(walker_name, walkers)
    elif len(F.parameters)>walkers.shape[-1]:
        newshape = list(walkers.shape)
        newshape[-1] = len(F.parameters)
        newwalkers = np.zeros(newshape, dtype=walkers.dtype)
        newwalkers[...,:walkers.shape[-1]] = walkers
        randshape = list(walkers.shape)
        randshape[-1] = len(F.parameters)-walkers.shape[-1]
        newwalkers[...,walkers.shape[-1]:] = (
            1e-7*np.random.randn(*randshape))
        walkers = newwalkers
        np.save(walker_name, walkers)
        

## Select and check a fitter configuration

## Rearrange the walkers

## Manage qsub status

Check whether a job is running, start it if it isn't, inspect what went wrong if it died.

See also [Ganglia](http://localhost:9080/ganglia/?c=nimrod&m=load_one&r=hour&s=by%20name&hc=4&mc=2)

In [146]:
o = subprocess.check_output(["qstat","-f"])
jobs = {}
cj = None
ce = None
for l in o.split("\n"):
    if l.startswith("\t"):
        jobs[cj][ce] += l[1:]
    elif l.startswith(" "*4):
        k, v = l.split("=", 1)
        k = k.strip()
        v = v.strip()
        ce = k
        jobs[cj][ce] = v
    elif l.startswith("Job Id:"):
        i, j = l.split(":",1)
        cj = j.strip()
        if cj in jobs:
            raise ValueError
        jobs[cj] = {}
    elif l.strip():
        print repr(l)

myjobs = {k:v for (k,v) in jobs.items() 
          if v["Job_Owner"]=="aarchiba@nimrod.cv.nrao.edu"}
for k in sorted(myjobs.keys()):
    j = myjobs[k]
    print k, j.get("job_state",None)

'},'
'},'
4164.nimrod.cv.nrao.edu C
4182.nimrod.cv.nrao.edu C


In [147]:
o = subprocess.check_output(["qsub", "submit_emcee_mpi.sh"])
o = o.strip()
print o
qsub_job_name_full = o
qsub_job_number = o.split(".")[0]

4183.nimrod.cv.nrao.edu


In [91]:
#myjobs

In [92]:
def uniquify(l):
    s = set()
    for i in l:
        if i not in s:
            yield i
            s.add(i)


In [94]:
for k in sorted(myjobs.keys()):
    print
    print k
    j = myjobs[k]
    p = j["Output_Path"].split(":")[1]
    if os.path.exists(p):
        for l in uniquify(open(p,"rt").readlines()):
            print l,
    print
    p = j["Error_Path"].split(":")[1]
    if os.path.exists(p):
        for l in uniquify(open(p,"rt").readlines()):
            print l,
    print
    


3544.nimrod.cv.nrao.edu

/usr/local/lib/python2.7/site-packages/stevedore/extension.py:17: UserWarning: Module mpl_toolkits was already imported from None, but /home/rhlocal/rhel6/x86_64/stow/Python-2.7.3/lib/python2.7/site-packages/matplotlib-1.2.1-py2.7-linux-x86_64.egg is being added to sys.path
  import pkg_resources



In [81]:
for k in sorted(myjobs.keys()):
    print myjobs[k]["exec_host"]

nimrod06/0
